# Monkeypox KG

# Package installation is required only once

In [ ]:
#pip install -r packages.txt

In [13]:
import pybel
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import MicroRna
from pybel.dsl import Rna
from pybel.dsl import Fragment
from tqdm import tqdm
import pandas as pd
from pybel.dsl import *
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from mpox import *
import pubchempy as pcp
import urllib
from urllib.parse import urlparse
import io
import seaborn as sns


# Drugs against monkeypox from https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12 + chembl database

In [17]:
mpox_chembl = pd.read_csv('data/normalized_data/chembl_mpox_actives.csv',sep=';',usecols=['Molecule ChEMBL ID','pChEMBL Value'])
mpox_chembl= mpox_chembl[mpox_chembl['pChEMBL Value'] >= 6]
mpox_chembl = list(mpox_chembl['Molecule ChEMBL ID'])
mpox_chembl

['CHEMBL1257073',
 'CHEMBL1257073',
 'CHEMBL1257073',
 'CHEMBL221054',
 'CHEMBL375768',
 'CHEMBL220150',
 'CHEMBL1257073',
 'CHEMBL220610',
 'CHEMBL1257073']

# Reading file from pubchem for monkeypox 
# https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244

In [18]:

mpox_pchem = pd.read_csv('data/normalized_data/TaxID_10244_bioactivity.csv')
mpox_pchem = mpox_pchem['cid']
mpox_pchem = list(set(mpox_pchem))


In [19]:
pchem2chembl_list = cid2chembl(mpox_pchem)

Pubchem: 134406 Converted to: CHEMBL321500
Pubchem: 44420748 Converted to: CHEMBL375768
Pubchem: 101879053 Converted to: CHEMBL3357216
Pubchem: 101879054 Converted to: CHEMBL3357215
Pubchem: 16124688 Converted to: CHEMBL1257073
Pubchem: 11243799 Converted to: CHEMBL220150
Pubchem: 155545375 Converted to: CHEMBL4565849
Pubchem: 11773602 Converted to: CHEMBL220610
Pubchem: 73124 Converted to: CHEMBL494759
Pubchem: 37542 Converted to: CHEMBL1643
Pubchem: 44341289 Converted to: CHEMBL114867
Pubchem: 44341811 Converted to: CHEMBL113930
Pubchem: 44341812 Converted to: CHEMBL326139
Pubchem: 118895042 Converted to: CHEMBL3753637
Pubchem: 118895044 Converted to: CHEMBL3753489
Pubchem: 60613 Converted to: CHEMBL152
Pubchem: 11301322 Converted to: CHEMBL220098
Pubchem: 189926 Converted to: CHEMBL112159
Pubchem: 5271015 Converted to: CHEMBL360506
Pubchem: 11452011 Converted to: CHEMBL219691
Pubchem: 11463022 Converted to: CHEMBL221054
Pubchem: 155521397 Converted to: CHEMBL4449833
Pubchem: 4511103

In [20]:
z = set(mpox_chembl).intersection(pchem2chembl_list)
z

{'CHEMBL1257073',
 'CHEMBL220150',
 'CHEMBL220610',
 'CHEMBL221054',
 'CHEMBL375768'}

In [21]:
chemblid = mpox_chembl+pchem2chembl_list
len(chemblid)

33

In [22]:
chemblid = set(chemblid)
chemblid = list(chemblid)
len(chemblid)

24

In [35]:
chembl2mech = RetMech(chemblid)

[]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[]
[]
[]
[]
[]
[{'mechanism_of_action': "Inosine-5'-monophosphate dehydrogenase 1 inhibitor", 'target_chembl_id': 'CHEMBL1822'}, {'mechanism_of_action': 'Hepatitis C virus NS5B RNA-dependent RNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL5375'}, {'mechanism_of_action': 'RNA inhibitor', 'target_chembl_id': 'CHEMBL2363073'}]


In [36]:
chembl2dis = RetDrugInd(chemblid)

[]
[]
[]
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Smallpox'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[]
[{'mesh_heading': 'Leukemia, Myeloid, Acute'}, {'mesh_heading': 'Hepatitis C, Chronic'}, {'mesh_heading': 'Hepatitis C'}, {'mesh_heading': 'Hepatitis B, Chronic'}, '...(remaining elements truncated)...']


In [33]:
#Returns a dictionary
def RetAct(chemblIds,j):
    GetAct = new_client.activity
    ActList = []
    #for i in range(len(chemblIds)):
    #print(chemblIds[0])
    for i in range(len(chemblIds)):
        filtered_list=['assay_chembl_id','assay_type','pchembl_value','target_chembl_id','target_organism','bao_label','target_type']
        acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,assay_type_iregex='(B|F)',target_organism='Homo sapiens').only(filtered_list)
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,target_organism='Homo sapiens').only(filtered_list)
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False)
        j=j+1

        #acts = [d for d in acts if d.get('target_organism') == 'Homo sapiens']
        #acts = [d for d in acts if d.get('bao_label') == 'single protein format']
        #acts = [d for d in acts if d.get('type') in ['Ki', 'IC50']]
        acts = [d for d in acts if float(d.get('pchembl_value')) >= 6]
        print(j)
        #print(len(acts))
        #acts = acts[:5]
        print(acts)
        ActList.append(list(acts))
    #print(ActList)
    named_ActList = dict(zip(chemblIds,ActList))
    named_ActList = {k: v for k, v in named_ActList.items() if v}
    return(named_ActList)

In [37]:
#chemblid = ['CHEMBL3294635','CHEMBL221054']
chembl2act = RetAct(chemblid,0)


1
[]
2
[{'assay_chembl_id': 'CHEMBL3294635', 'assay_type': 'B', 'bao_label': 'single protein format', 'pchembl_value': '7.62', 'target_chembl_id': 'CHEMBL2664', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '24.0'}, {'assay_chembl_id': 'CHEMBL3294636', 'assay_type': 'B', 'bao_label': 'cell-based format', 'pchembl_value': '8.03', 'target_chembl_id': 'CHEMBL2664', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '9.4'}]
3
[]
4
[]
5
[]
6
[]
7
[]
8
[]
9
[]
10
[]
11
[]
12
[]
13
[]
14
[]
15
[{'assay_chembl_id': 'CHEMBL3757541', 'assay_type': 'B', 'bao_label': 'single protein format', 'pchembl_value': '8.49', 'target_chembl_id': 'CHEMBL3751646', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '3.2'}]
16
[]
17
[]
18
[{'assay_chembl_id': 'CHEMBL746888', 'assay_type': 'F', 'bao_label': 'cell-based format', 'pchembl_value': '6.30', 'target_chembl_id': 'CHEMBL614200', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '0.5'}, {'assay_chembl_id': 'CHEMB

In [10]:
prtn_as_chembl = Ret_chembl_protein(chembl2act) + Ret_chembl_protein(chembl2mech)
prtn_as_chembl = set(prtn_as_chembl)
prtn_as_chembl = list(prtn_as_chembl)
prtn_as_chembl

['CHEMBL614200',
 'CHEMBL614149',
 'CHEMBL1872',
 'CHEMBL613888',
 'CHEMBL1822',
 'CHEMBL3988586',
 'CHEMBL340',
 'CHEMBL2664',
 'CHEMBL4296170',
 'CHEMBL1293235',
 'CHEMBL2363073',
 'CHEMBL5375',
 'CHEMBL3751646']

In [11]:
chembl2uprot = chembl2uniprot(prtn_as_chembl,0)

old 9
newLen 7
2


In [12]:
chembl2uprot

{'CHEMBL1822': [{'xref_id': 'R-HSA-6798695',
   'xref_name': 'Neutrophil degranulation.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-73817',
   'xref_name': 'Purine ribonucleoside monophosphate biosynthesis.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-9679191',
   'xref_name': 'Potential therapeutics for SARS.',
   'xref_src_db': 'Reactome'},
  {'accession': 'P20839'},
  {'component_synonym': 'IMPDH1', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL3988586': [{'accession': 'P06856'},
  {'component_synonym': 'POL', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL340': [{'xref_id': 'R-HSA-211981',
   'xref_name': 'Xenobiotics.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5423646',
   'xref_name': 'Aflatoxin activation and detoxification.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-9027307',
   'xref_name': 'Biosynthesis of maresin-like SPMs.',
   'xref_src_db': 'Reactome'},
  {'accession': 'P08684'},
  {'component_synonym': 'CYP3A4', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL

In [13]:
#update chembl protein nodes with gene symbol
#use previously created variable that stores activity info
chembl2act = chembl2gene2path(chembl2uprot,chembl2act)
chembl2mech = chembl2gene2path(chembl2uprot,chembl2mech)

In [111]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [112]:
mpox_graph = chem2moa_rel_2(chembl2mech,'HGNC',mpox_graph)
mpox_graph = chem2dis_rel(chembl2dis,mpox_graph)
mpox_graph = chem2act_rel_2(chembl2act,'HGNC',mpox_graph)
mpox_graph = chem2gene2path_rel(chembl2uprot,'HGNC',mpox_graph)
#to_jupyter(mpox_graph)

In [ ]:
# filename = ('data/normalized_data/monkeypox_pickle2')
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)

In [ ]:
#import monkey pox graph
infile = open('data/normalized_data/monkeypox_pickle2','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [113]:
mpox_graph.summarize

Name,Monkeypox Graph
Version,
Nodes,103
Namespaces,6
Edges,218
Annotations,0
Citations,1
Authors,0
Components,3
Warnings,0
Network Density,0.0208


In [18]:
#Fetch all proteins from the graph
chemblProt = Ret_uprotid(chembl2uprot)
chemblProt


['P20839', 'P06856', 'P08684', 'P23526', 'P02545', 'Q8JXU8', 'O43865']

# Working with monkeypox proteins from uniprot

In [32]:
mpox_prot_df = pd.read_excel('data/normalized_data/uniprot-taxonomy Monkeypox+virus+[10244] -filtered-reviewed yes.xlsx')
mpox_prot= list(mpox_prot_df['Entry'])
mpox_prot

C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['Q8V4Y0',
 'Q8V4S4',
 'Q8V571',
 'P04363',
 'Q8V4V3',
 'Q8V4U9',
 'Q8V4T3',
 'Q8V518',
 'Q8V4V4',
 'Q8V4T7',
 'Q8V566']

# Genes from Malacards

In [19]:
mpox_genes_df = pd.read_excel('data/normalized_data/malacardsGenes.xlsx')

In [20]:
mpox_genes = list(mpox_genes_df['Entry'])
len(mpox_genes)

13

In [21]:
uprots = mpox_genes+chemblProt

In [22]:
uprots

['P0C0L4',
 'P0C0L5',
 'Q9Y258',
 'Q07444',
 'P01730',
 'P01732',
 'Q8NH81',
 'P26718',
 'P08174',
 'P00387',
 'P15529',
 'Q14653',
 'P01562',
 'P20839',
 'P06856',
 'P08684',
 'P23526',
 'P02545',
 'Q8JXU8',
 'O43865']

In [23]:
uprots_ext = ExtractFromUniProt(uprots)

https://www.uniprot.org/uniprot/P0C0L4.txt
{'Gene': 'C4A'}
https://www.uniprot.org/uniprot/P0C0L5.txt
{'Gene': 'C4B'}
https://www.uniprot.org/uniprot/Q9Y258.txt
https://www.uniprot.org/uniprot/Q07444.txt
{'Gene': 'KLRC3'}
https://www.uniprot.org/uniprot/P01730.txt
{'Gene': 'CD4'}
https://www.uniprot.org/uniprot/P01732.txt
{'Gene': 'CD8A'}
https://www.uniprot.org/uniprot/Q8NH81.txt
{'Gene': 'OR10G6'}
https://www.uniprot.org/uniprot/P26718.txt
{'Gene': 'KLRK1'}
https://www.uniprot.org/uniprot/P08174.txt
{'Gene': 'CD55'}
https://www.uniprot.org/uniprot/P00387.txt
https://www.uniprot.org/uniprot/P15529.txt
{'Gene': 'CD46'}
https://www.uniprot.org/uniprot/Q14653.txt
https://www.uniprot.org/uniprot/P01562.txt
{'Gene': 'IFNA1'}
https://www.uniprot.org/uniprot/P20839.txt
https://www.uniprot.org/uniprot/P06856.txt
{'Gene': 'POL'}
https://www.uniprot.org/uniprot/P08684.txt
https://www.uniprot.org/uniprot/P23526.txt
{'Gene': 'AHCY'}
https://www.uniprot.org/uniprot/P02545.txt
{'Gene': 'LMNA'}
http

In [33]:
mpox_prot_ext = ExtractFromUniProt(mpox_prot)

https://www.uniprot.org/uniprot/Q8V4Y0.txt
https://www.uniprot.org/uniprot/Q8V4S4.txt
https://www.uniprot.org/uniprot/Q8V571.txt
{'Gene': 'p28'}
https://www.uniprot.org/uniprot/P04363.txt
{'Gene': 'TK'}
https://www.uniprot.org/uniprot/Q8V4V3.txt
{'Gene': 'RPO132'}
https://www.uniprot.org/uniprot/Q8V4U9.txt
https://www.uniprot.org/uniprot/Q8V4T3.txt
https://www.uniprot.org/uniprot/Q8V518.txt
https://www.uniprot.org/uniprot/Q8V4V4.txt
{'Gene': 'VITF3L'}
https://www.uniprot.org/uniprot/Q8V4T7.txt
https://www.uniprot.org/uniprot/Q8V566.txt


In [34]:
mpox_prot_ext

{'Q8V4Y0': {'Disease': {},
  'Reactome': {},
  'Function': {'carbonate dehydratase activity': 'GO:0004089',
   'zinc ion binding': 'GO:0008270'},
  'BioProcess': {'viral entry into host cell': 'GO:0046718',
   'virion attachment to host cell': 'GO:0019062'},
  'Gene': {}},
 'Q8V4S4': {'Disease': {},
  'Reactome': {},
  'Function': {"2',3'-cyclic GMP-AMP binding": 'GO:0061507',
   'nuclease activity': 'GO:0004518'},
  'BioProcess': {'suppression by virus of host innate immune response': 'GO:0039503'},
  'Gene': 'B4R'},
 'Q8V571': {'Disease': {},
  'Reactome': {},
  'Function': {'acid-amino acid ligase activity': 'GO:0016881',
   'metal ion binding': 'GO:0046872',
   'transferase activity': 'GO:0016740'},
  'BioProcess': {'modulation by virus of host apoptotic process': 'GO:0039526',
   'modulation by virus of host protein ubiquitination': 'GO:0039648'},
  'Gene': 'p28'},
 'P04363': {'Disease': {},
  'Reactome': {},
  'Function': {'ATP binding': 'GO:0005524',
   'metal ion binding': 'GO:

In [30]:

# function to create relationships for dict created using ExtractFromUniProt
def uniprot_rel(named_uprotList,org,itmpGraph):
    for item in named_uprotList:
        # print(named_uprotList[item]['Function'].keys())
        fun = list(named_uprotList[item]['Function'].keys())
        bp = list(named_uprotList[item]['BioProcess'].keys())
        for f in fun:
            if str(named_uprotList[item]['Gene']) != 'nan' and not isinstance(named_uprotList[item]['Gene'], dict):
                itmpGraph.add_association(Protein(namespace=org, name=named_uprotList[item]['Gene']),
                                          BiologicalProcess(namespace='GOMF', name=f),
                                          citation='UniProt database', evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace=org, name=item),
                                          BiologicalProcess(namespace='GOMF', name=f),
                                          citation='UniProt database', evidence='UniProt query')

        for b in bp:
            if str(named_uprotList[item]['Gene']) != 'nan' and not isinstance(named_uprotList[item]['Gene'], dict):
                itmpGraph.add_association(Protein(namespace=org, name=named_uprotList[item]['Gene']),
                                          BiologicalProcess(namespace='GOBP', name=b),
                                          citation='UniProt database', evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace=org, name=item),
                                          BiologicalProcess(namespace='GOBP', name=b),
                                          citation='UniProt database', evidence='UniProt query')

    return (itmpGraph)


In [114]:
mpox_graph = uniprot_rel(uprots_ext,'HGNC',mpox_graph)
mpox_graph = uniprot_rel(mpox_prot_ext,'MPXV',mpox_graph)
#to_jupyter(mpox_graph)

In [36]:
# filename = 'data/normalized_data/monkeypox_pickle_25thJuly'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)

In [70]:
#import monkey pox graph
infile = open('data/normalized_data/monkeypox_pickle_25thjuly','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [ ]:
to_jupyter(mpox_graph)

In [15]:
def _get_target_data(protein_list: list, organism: str):
    """Get chemical for target data from ChEMBL"""
    df_data = []

    target = new_client.target
    activity = new_client.activity

    for protein in protein_list:
        if pd.isna(protein):
            continue
        try:
            prot_data = [target.search(protein)[0]]

            # Search for protein with same synonym
            if prot_data == [None]:
                prot_data = target.filter(
                    target_synonym__icontains=protein, target_organism__istartswith=organism
                ).only(['target_chembl_id', 'target_pref_name', 'molecule_chembl_id', 'molecule_pref_name'])
        except HttpBadRequest:
            print(f'Cannot search for {protein} due to chembl error')
            continue

        # No results found
        if not prot_data:
            continue

        for prot in tqdm(prot_data, f'Analying data for {protein}'):
            # Absence of chembl id
            if not prot['target_chembl_id']:
                continue

            prot_activity_data = activity.filter(
                target_chembl_id=prot['target_chembl_id'],
                assay_type_iregex='(B|F)',
            ).only([
                'pchembl_value', 'molecule_chembl_id', 'activity_id', 'target_pref_name', 'molecule_pref_name'
            ])

            if len(prot_activity_data) < 1:
                continue

            for i in prot_activity_data:
                tmp = {}

                if i['pchembl_value'] is None:
                    continue

                pchembl_val = i['pchembl_value']

                if float(pchembl_val) < 6:
                    tmp['activity'] = 'inhibitor'
                else:
                    tmp['activity'] = 'activator'

                tmp['protein_symbol'] = protein
                tmp['protein_name'] = i['target_pref_name']
                tmp['aid'] = str(i['activity_id'])
                tmp['chembl_id'] = i['molecule_chembl_id']
                tmp['compound_name'] = i['molecule_pref_name'].capitalize() if i['molecule_pref_name'] else ''
                df_data.append(tmp)

    # Merge duplicated protein-chemical entries into one
    df = pd.DataFrame()

    for idx, row in tqdm(enumerate(df_data), total=len(df_data), desc='Preparing data'):
        if idx == 0:
            df = df.append(row, ignore_index=True)
        else:
            _in_df = df.loc[
                (df['protein_symbol'] == row['protein_symbol']) & (df['chembl_id'] == row['chembl_id'])
            ]

            if _in_df.empty:
                df = df.append(row, ignore_index=True)
            else:
                row_index = _in_df.index

                # Check existing citations
                existing_assays = set(df.loc[row_index, 'aid'].values[0].split(' | '))
                old_count = len(existing_assays)
                existing_assays.add(row['aid'])
                new_count = len(existing_assays)

                # Check if new citation added, if yes - add respective data
                if old_count < new_count:
                    df.loc[row_index, 'aid'] = ' | '.join(existing_assays)
    df = df[['activity', 'protein_symbol', 'protein_name', 'aid', 'chembl_id', 'compound_name']]
    return df

def target_list_to_chemical(
    proteins: list,
    organism: str = 'Homo sapiens',
    output_dir: str = ''
) -> None:
    """Extract chemical information on list of targets
    Usage:
    >> target_list_to_chemical(proteins=['RIPK'])
    """

    df = _get_target_data(protein_list=proteins, organism=organism)
    #os.makedirs(output_dir, exist_ok=True)
    #df.to_csv(os.path.join(output_dir, 'chemical_annotated.csv'), sep='\t', index=False)
    return(df)


In [16]:
uprot2chem = target_list_to_chemical(uprots)

Analying data for Q8JXU8:   0%|          | 0/1 [01:07<?, ?it/s]


HttpApplicationError: Error for url https://www.ebi.ac.uk/chembl/api/data/activity.json, server response: <!doctype html>
<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!-- Consider adding an manifest.appcache: h5bp.com/d/Offline -->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
<head>
    <meta charset="utf-8">

    <!-- Use the .htaccess and remove these lines to avoid edge case issues.
 More info: h5bp.com/b/378 -->
    <!-- <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"> --> <!-- Not yet implemented -->

    <title>Server error &lt; EMBL-EBI</title>
    <meta name="description" content="EMBL-EBI"><!-- Describe what this page is about -->
    <meta name="keywords" content="bioinfor

In [ ]:
uprot2chem

In [ ]:
# filename = 'data/normalized_data/uprot2chembl'
# outfile = open(filename,'wb')
# pickle.dump(uprot2chem,outfile)

In [19]:
infile = open('data/normalized_data/uprot2chembl','rb')
uprot2chem = pickle.load(infile)
infile.close()

In [20]:
uprot2chem.to_csv('data/normalized_data/uprot2chembl.csv',sep=',')

In [10]:
uprot2chem=set(uprot2chem['protein_symbol'])
uprot2chem

{'O43865',
 'P00387',
 'P01730',
 'P02545',
 'P06856',
 'P08684',
 'P20839',
 'P23526',
 'Q14653',
 'Q8JXU8'}

In [11]:
uprots= ['Q8NH81',
 'Q68D06',
 'Q14653',
 'Q8WVL7',
 'Q9Y258',
 'Q8IYM2',
 'P01732',
 'P00387',
 'P0C0L4',
 'P01730',
 'Q05823',
 'P0C0L5',
 'P01562',
 'P07315',
 'P15529',
 'P26718',
 'P08174',
 'P23919',
 'P01574',
 'P20839',
 'P23526',
 'O43865',
 'Q8JXU8',
 'P08684']

In [23]:
active = uprot2chem.loc[uprot2chem['activity'].str.contains('activator',na=False)]
active = active.reset_index(drop=True)
#active.to_csv('data/normalized_data/active_uprot2chem.csv',sep=',')

In [40]:
active

,activity,protein_symbol,protein_name,aid,chembl_id,compound_name
0,activator,P01730,T-cell surface antigen CD4,105411,CHEMBL506605,Isocomplestatin
1,activator,P01730,T-cell surface antigen CD4,106547,CHEMBL525803,Chloropeptin
2,activator,P01730,T-cell surface antigen CD4,106550,CHEMBL3143464,
3,activator,P01730,T-cell surface antigen CD4,20710180,CHEMBL1852155,
4,activator,P01730,T-cell surface antigen CD4,22435351,CHEMBL1819181,Cyclotriazadisulfonamide hydrochloride
...,...,...,...,...,...,...
9031,activator,P08684,Cytochrome P450 3A4,22851099,CHEMBL4739976,
9032,activator,P08684,Cytochrome P450 3A4,22851101,CHEMBL4760660,
9033,activator,P08684,Cytochrome P450 3A4,22851104,CHEMBL4758649,
9034,activator,P08684,Cytochrome P450 3A4,22851106,CHEMBL4788002,


In [42]:
active['compound_name'][3] == ''

True

In [25]:
active = active[active['compound_name'] != '']
active = active.reset_index(drop=True)
active.to_csv('data/normalized_data/active_uprot2chem.csv',sep=',')

In [44]:
chem = active['chembl_id']

In [ ]:
len(chem)

In [ ]:
chem = ['CHEMBL506605',
 'CHEMBL525803',
 'CHEMBL3143464',
 'CHEMBL3138360',
 'CHEMBL3138345',
 'CHEMBL3138187',
 'CHEMBL3138374',
 'CHEMBL3138095',
 'CHEMBL3138378',
 'CHEMBL3138119',
 'CHEMBL231224',
 'CHEMBL267655',
 'CHEMBL388209',
 'CHEMBL230799',
 'CHEMBL267658',
 'CHEMBL388208',
 'CHEMBL267657',
 'CHEMBL230798',
 'CHEMBL388207',
 'CHEMBL230796',
 'CHEMBL1852155',
 'CHEMBL1819181',
 'CHEMBL4783030',
 'CHEMBL4755141',
 'CHEMBL4783761',
 'CHEMBL4750160',
 'CHEMBL4743003',
 'CHEMBL4782080',
 'CHEMBL4746252',
 'CHEMBL4747494',
 'CHEMBL4758659',
 'CHEMBL4750983',
 'CHEMBL4786809',
 'CHEMBL4748787',
 'CHEMBL4785236',
 'CHEMBL4791666',
 'CHEMBL4525466',
 'CHEMBL4471585']

In [ ]:
chem2dis = RetDrugInd(chem)

In [ ]:
chem2mech = RetMech(chem)

In [ ]:
chem2act = RetAct(chem,0)

In [ ]:
chembl2actmechdis = {}
chembl2actmechdis['activity'] = chem2act
chembl2actmechdis['mechanism'] = chem2mech
chembl2actmechdis['disease']= chem2dis

In [ ]:
# filename = 'data/normalized_data/chembl2actmechdis'
# outfile = open(filename,'wb')
# pickle.dump(chembl2actmechdis,outfile)

In [45]:
infile = open('data/normalized_data/chembl2actmechdis','rb')
chembl2actmechdis = pickle.load(infile)
infile.close()

In [46]:
chem2act = chembl2actmechdis['activity']
chem2mech = chembl2actmechdis['mechanism']
chem2dis = chembl2actmechdis['disease']

In [47]:
chemid2 = Ret_chembl_protein(chem2act) + Ret_chembl_protein(chem2mech)
chemid2

['CHEMBL1824',
 'CHEMBL1791',
 'CHEMBL1942',
 'CHEMBL217',
 'CHEMBL3959',
 'CHEMBL614649',
 'CHEMBL227',
 'CHEMBL4956',
 'CHEMBL1293298',
 'CHEMBL1867',
 'CHEMBL614358',
 'CHEMBL2111463',
 'CHEMBL2071',
 'CHEMBL614274',
 'CHEMBL209',
 'CHEMBL1803',
 'CHEMBL2392',
 'CHEMBL385',
 'CHEMBL613107',
 'CHEMBL249',
 'CHEMBL262',
 'CHEMBL325',
 'CHEMBL5463',
 'CHEMBL1293267',
 'CHEMBL4685',
 'CHEMBL3401',
 'CHEMBL2094257',
 'CHEMBL1293255',
 'CHEMBL3923',
 'CHEMBL1697668',
 'CHEMBL1293235',
 'CHEMBL2421',
 'CHEMBL1908',
 'CHEMBL1875',
 'CHEMBL3227',
 'CHEMBL1822',
 'CHEMBL4681',
 'CHEMBL4523354',
 'CHEMBL614735',
 'CHEMBL1293249',
 'CHEMBL1936',
 'CHEMBL3729',
 'CHEMBL399',
 'CHEMBL5112',
 'CHEMBL2093872',
 'CHEMBL3746',
 'CHEMBL2093861',
 'CHEMBL2023',
 'CHEMBL613517',
 'CHEMBL614530',
 'CHEMBL2002',
 'CHEMBL3577',
 'CHEMBL340',
 'CHEMBL2903',
 'CHEMBL2094132',
 'CHEMBL1871',
 'CHEMBL221',
 'CHEMBL1898',
 'CHEMBL4296429',
 'CHEMBL1801',
 'CHEMBL614067',
 'CHEMBL3832943',
 'CHEMBL614721',
 'CHE

In [48]:
chem2uni = chembl2uniprot(chemid2,0)

old 372
newLen 367
5


In [59]:
chem2uni

{'CHEMBL1824': [{'xref_id': 'R-HSA-1227986',
   'xref_name': 'Signaling by ERBB2.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1250196',
   'xref_name': 'SHC1 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1251932',
   'xref_name': 'PLCG1 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1257604',
   'xref_name': 'PIP3 activates AKT signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1306955',
   'xref_name': 'GRB7 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1358803',
   'xref_name': 'Downregulation of ERBB2:ERBB3 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1963640',
   'xref_name': 'GRB2 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1963642',
   'xref_name': 'PI3K events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2219530',
   'xref_name': 'Constitutive Signaling by Aberrant PI3

In [53]:
protList = Ret_uprotid(chem2uni)
len(protList)

309

In [54]:
protList_ext = ExtractFromUniProt(protList)

https://www.uniprot.org/uniprot/P04626.txt
{'Gene': 'ERBB2'}
https://www.uniprot.org/uniprot/P37088.txt
{'Gene': 'SCNN1A'}
https://www.uniprot.org/uniprot/P18089.txt
{'Gene': 'ADRA2B'}
https://www.uniprot.org/uniprot/P14416.txt
{'Gene': 'DRD2'}
https://www.uniprot.org/uniprot/P16083.txt
{'Gene': 'NQO2'}
https://www.uniprot.org/uniprot/P30556.txt
https://www.uniprot.org/uniprot/P47869.txt
{'Gene': 'GABRA2'}
https://www.uniprot.org/uniprot/Q01453.txt
{'Gene': 'PMP22'}
https://www.uniprot.org/uniprot/P08913.txt
https://www.uniprot.org/uniprot/Q9UBS5.txt
{'Gene': 'GABBR1'}
https://www.uniprot.org/uniprot/Q09428.txt
{'Gene': 'ABCC8'}
https://www.uniprot.org/uniprot/P07477.txt
https://www.uniprot.org/uniprot/P20701.txt
https://www.uniprot.org/uniprot/P06746.txt
{'Gene': 'POLB'}
https://www.uniprot.org/uniprot/P25103.txt
{'Gene': 'TACR1'}
https://www.uniprot.org/uniprot/P49841.txt
{'Gene': 'GSK3B'}
https://www.uniprot.org/uniprot/Q13547.txt
https://www.uniprot.org/uniprot/Q13510.txt
https://w

https://www.uniprot.org/uniprot/P09917.txt
https://www.uniprot.org/uniprot/P25098.txt
https://www.uniprot.org/uniprot/Q99720.txt
{'Gene': 'SIGMAR1'}
https://www.uniprot.org/uniprot/P35368.txt
{'Gene': 'ADRA1B'}
https://www.uniprot.org/uniprot/P50579.txt
https://www.uniprot.org/uniprot/P50406.txt
{'Gene': 'HTR6'}
https://www.uniprot.org/uniprot/Q04828.txt
{'Gene': 'AKR1C1'}
https://www.uniprot.org/uniprot/B2RXH2.txt
{'Gene': 'KDM4E'}
https://www.uniprot.org/uniprot/Q00535.txt
https://www.uniprot.org/uniprot/P08235.txt
{'Gene': 'NR3C2'}
https://www.uniprot.org/uniprot/Q14534.txt
{'Gene': 'SQLE'}
https://www.uniprot.org/uniprot/P08588.txt
https://www.uniprot.org/uniprot/P02708.txt
{'Gene': 'CHRNA1'}
https://www.uniprot.org/uniprot/P37058.txt
https://www.uniprot.org/uniprot/P17787.txt
{'Gene': 'CHRNB2'}
https://www.uniprot.org/uniprot/P15559.txt
https://www.uniprot.org/uniprot/P01730.txt
{'Gene': 'CD4'}
https://www.uniprot.org/uniprot/Q01959.txt
{'Gene': 'SLC6A3'}
https://www.uniprot.org/u

https://www.uniprot.org/uniprot/P05230.txt
{'Gene': 'FGF1'}
https://www.uniprot.org/uniprot/P01112.txt
{'Gene': 'HRAS'}
https://www.uniprot.org/uniprot/P20309.txt
{'Gene': 'CHRM3'}
https://www.uniprot.org/uniprot/P43088.txt
{'Gene': 'PTGFR'}
https://www.uniprot.org/uniprot/E0VCW8.txt
https://www.uniprot.org/uniprot/P68371.txt
{'Gene': 'TUBB4B'}
https://www.uniprot.org/uniprot/P10613.txt
https://www.uniprot.org/uniprot/P04818.txt
https://www.uniprot.org/uniprot/P21917.txt
{'Gene': 'DRD4'}
https://www.uniprot.org/uniprot/P35348.txt
{'Gene': 'ADRA1A'}


In [55]:
# filename = 'data/normalized_data/allprot_fromchembl'
# outfile = open(filename,'wb')
# pickle.dump(protList_ext,outfile)

In [57]:
infile = open('data/normalized_data/allprot_fromchembl','rb')
allprot = pickle.load(infile)
infile.close()

In [76]:
#mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [115]:
mpox_graph = uniprot_rel(allprot,'HGNC',mpox_graph)
pchem_act_new= chembl2gene2path(chem2uni,chem2act)
pchem_mech_new = chembl2gene2path(chem2uni,chem2mech)
mpox_graph = chem2act_rel_2(pchem_act_new,'HGNC',mpox_graph)
mpox_graph = chem2moa_rel_2(pchem_mech_new,'HGNC',mpox_graph)
mpox_graph = chem2dis_rel(chem2dis,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,'HGNC',mpox_graph)
#to_jupyter(mpox_graph)

In [ ]:
chem2uni['CHEMBL2094125'][2]['component_synonym']

In [82]:
# filename = 'data/normalized_data/monkeypox_pickle_final_25thJuly'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)
# outfile.close()

In [74]:
infile = open('data/normalized_data/monkeypox_pickle_final_25thJuly','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [116]:
mpox_graph.summarize

Name,Monkeypox Graph
Version,
Nodes,4916
Namespaces,9
Edges,20962
Annotations,0
Citations,2
Authors,0
Components,3
Warnings,0
Network Density,0.000868


In [85]:
chembl_id = []
for node in mpox_graph.nodes():
    if isinstance(node,pybel.dsl.Abundance):
        if node.namespace == 'ChEMBL':
            chembl_id.append(node.name)
            
    

In [86]:
len(chembl_id)

565

In [117]:
def chembl2rxn_rel(itmpGraph):
    
    infile = open('data/normalized_data/drugReactions.pkl','rb')
    rxn_df = pickle.load(infile)
    infile.close()
    
    chembl_id = []
    for node in itmpGraph.nodes():
        if isinstance(node,pybel.dsl.Abundance):
            if node.namespace == 'ChEMBL':
                chembl_id.append(node.name)
            
    chembl_id_rxn = rxn_df[rxn_df['chembl_id'].isin(chembl_id)]
    chembl_id_rxn = chembl_id_rxn.reset_index(drop=True)
    for i in range(len(chembl_id_rxn)):
        itmpGraph.add_association(Abundance(namespace='ChEMBL',name = chembl_id_rxn['chembl_id'][i]),
                                  Pathology(namespace='SideEffect',name = chembl_id_rxn['event'][i]),
                                  citation = "OpenTargets Platform",evidence = 'DrugReactions')
        
    return(itmpGraph)
                                                                                            
    

In [118]:
mpox_graph = chembl2rxn_rel(mpox_graph)

In [119]:
mpox_graph.summarize

Name,Monkeypox Graph
Version,
Nodes,8235
Namespaces,10
Edges,40422
Annotations,0
Citations,3
Authors,0
Components,3
Warnings,0
Network Density,0.000596


In [121]:
filename = 'data/normalized_data/monkeypox_pickle_final_25thJuly.pkl'
outfile = open(filename,'wb')
pickle.dump(mpox_graph,outfile)
outfile.close()

In [5]:
infile = open('data/normalized_data/monkeypox_pickle_final_25thJuly.pkl','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [92]:
test = mpox_graph.summarize.nodes()
test

Type (4)             Count  Example
-----------------  -------  ------------------------------------------------
Pathology             4226  path(SideEffect:"endometrial cancer")
BiologicalProcess     2195  bp(GOBP:"cellular response to glucose stimulus")
Abundance             1427  a(ChEMBL:CHEMBL1514715)
Protein                387  p(HGNC:BLM)


In [ ]:
mpox_graph

In [3]:
infile = open('data/normalized_data/targets.pkl','rb')
targetability = pickle.load(infile)
infile.close()

In [6]:
protein = []
for node in mpox_graph.nodes():
    if isinstance(node,pybel.dsl.Protein):
        if node.namespace == 'HGNC':
            protein.append(node.name)
            
            

In [7]:
len(protein)

377

In [8]:
targetability = targetability[targetability['approvedSymbol'].isin(protein)]
targetability = targetability.reset_index(drop=True)

In [9]:
#targetability[targetability['approvedSymbol']=='C1R']
targetability.dropna(subset=['tractability'],inplace=True)
targetability = targetability.reset_index(drop=True)
targetability


,id,approvedSymbol,biotype,transcriptIds,genomicLocation,alternativeGenes,approvedName,go,hallmarks,synonyms,...,obsoleteNames,constraint,tep,proteinIds,dbXrefs,chemicalProbes,homologues,tractability,safetyLiabilities,pathways
0,ENSG00000115705,TPO,protein_coding,"[ENST00000650224, ENST00000382269, ENST0000032...","{'chromosome': '2', 'start': 1374066, 'end': 1...",None,thyroid peroxidase,"[{'id': 'GO:0098869', 'source': 'GO_REF:000010...",None,"[{'label': 'Thyroid peroxidase', 'source': 'un...",...,[],"[{'constraintType': 'syn', 'score': -0.8653900...",None,"[{'id': 'P07202', 'source': 'uniprot_swissprot...","[{'id': '12015', 'source': 'HGNC'}, {'id': 'CH...",None,"[{'speciesId': '9598', 'speciesName': 'Chimpan...","[{'modality': 'SM', 'id': 'Approved Drug', 'va...",None,"[{'pathwayId': 'R-HSA-209968', 'pathway': 'Thy..."
1,ENSG00000080824,HSP90AA1,protein_coding,"[ENST00000216281, ENST00000557089, ENST0000056...","{'chromosome': '14', 'start': 102080742, 'end'...",None,heat shock protein 90 alpha family class A mem...,"[{'id': 'GO:0005829', 'source': 'Reactome:R-HS...",None,"[{'label': 'Heat shock protein HSP 90-alpha', ...",...,"[{'label': 'heat shock 90kD protein 1, alpha',...","[{'constraintType': 'syn', 'score': -8.6224002...",None,"[{'id': 'P07900', 'source': 'uniprot_swissprot...","[{'id': '5253', 'source': 'HGNC'}, {'id': '1BY...","[{'control': None, 'drugId': 'CHEMBL467399', '...","[{'speciesId': '9606', 'speciesName': 'Human',...","[{'modality': 'SM', 'id': 'Approved Drug', 'va...",None,"[{'pathwayId': 'R-HSA-9009391', 'pathway': 'Ex..."
2,ENSG00000158748,HTR6,protein_coding,[ENST00000289753],"{'chromosome': '1', 'start': 19664875, 'end': ...",None,5-hydroxytryptamine receptor 6,"[{'id': 'GO:0030425', 'source': 'PMID:21873635...",None,"[{'label': '5-hydroxytryptamine receptor 6', '...",...,[{'label': '5-hydroxytryptamine (serotonin) re...,"[{'constraintType': 'syn', 'score': -0.9145399...",None,"[{'id': 'P50406', 'source': 'uniprot_swissprot...","[{'id': '5301', 'source': 'HGNC'}, {'id': 'CHE...",None,"[{'speciesId': '9606', 'speciesName': 'Human',...","[{'modality': 'SM', 'id': 'Approved Drug', 'va...","[{'event': 'receptor binding', 'eventId': None...","[{'pathwayId': 'R-HSA-418555', 'pathway': 'G a..."
3,ENSG00000123595,RAB9A,protein_coding,"[ENST00000464506, ENST00000618931, ENST0000024...","{'chromosome': 'X', 'start': 13689128, 'end': ...",None,"RAB9A, member RAS oncogene family","[{'id': 'GO:0042470', 'source': 'GO_REF:000002...",None,"[{'label': 'Ras-related protein Rab-9A', 'sour...",...,"[{'label': 'RAB9, member RAS oncogene family',...","[{'constraintType': 'syn', 'score': 0.95749002...",None,"[{'id': 'P51151', 'source': 'uniprot_swissprot...","[{'id': '9792', 'source': 'HGNC'}, {'id': '1WM...",None,"[{'speciesId': '9606', 'speciesName': 'Human',...","[{'modality': 'SM', 'id': 'Approved Drug', 'va...",None,"[{'pathwayId': 'R-HSA-8873719', 'pathway': 'RA..."
4,ENSG00000170425,ADORA2B,protein_coding,"[ENST00000304222, ENST00000582124]","{'chromosome': '17', 'start': 15945130, 'end':...",None,adenosine A2b receptor,"[{'id': 'GO:0005886', 'source': 'Reactome:R-HS...",None,"[{'label': 'Adenosine receptor A2b', 'source':...",...,[],"[{'constraintType': 'syn', 'score': 0.42638000...",None,"[{'id': 'P29275', 'source': 'uniprot_swissprot'}]","[{'id': '264', 'source': 'HGNC'}, {'id': 'CHEM...","[{'control': None, 'drugId': None, 'id': 'MRS1...","[{'speciesId': '9606', 'speciesName': 'Human',...","[{'modality': 'SM', 'id': 'Approved Drug', 'va...","[{'event': 'increased/decreased inflammation',...","[{'pathwayId': 'R-HSA-417973', 'pathway': 'Ade..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,ENSG00000244731,C4A,protein_coding,"[ENST00000460060, ENST00000470365, ENST0000046...","{'chromosome': '6', 'start': 31982057, 'end': ...","[ENSG00000227746, ENSG00000206340]",complement C4A (Rodgers blood group),"[{'id': 'GO:0005886', 'source': 'Reactome:R-HS...",None,"[{'label':

In [10]:
gene = []
flag = []
i = 0
j = 0
for i in range(len(targetability)):
    #j = j+1
    #print(j)
#     if not targetability['tractability'][i]:
#         continue
        
    #if pd.isna(targetability['tractability'][i].any() != True): 
    if targetability['tractability'][i][7]['value'] == True:
        #i = i+1
        #print(i)
        #print(targetability['tractability'][i])
        #print(targetability['approvedSymbol'][i])
        #print(targetability['tractability'][i][7]['value'])
        gene.append(targetability['approvedSymbol'][i])
        flag.append('Yes')
        
    else:
        print(targetability['approvedSymbol'][i])
        gene.append(targetability['approvedSymbol'][i])
        flag.append('No')

        

RGS17
C4B
C5
IFNA1
IRF3
KLRC3
SERPINA6
SLC7A11
CCL26
NAPRT
TMEM97
CD8A
CD4
CSNK2B
ASAH1
AHCYL1
FUT7
KLRK1
CD46
FABP3
CD55
OR10G6
CYB5R3
C4A
MEN1


In [11]:
druggability = pd.DataFrame()
druggability['Gene'] = gene
druggability['Druggable Family'] = flag
druggability.to_csv('data/normalized_data/druggability.csv',sep=',')

In [ ]:
if targetability['tractability'][144].any():
    print('ok')
    targetability['tractability'][144]


In [ ]:
if targetability['tractability'][144].any:
    print(targetability['tractability'][144])

In [ ]:
pd.isna(targetability['tractability'][144]) != True

In [ ]:
targetability['tractability'][144]


In [ ]:
pybel.to_graphml(mpox_graph,'data/normalized_data/Monkeypox_KG_20thJuly.graphml')


In [ ]:
sns.set_theme(style="whitegrid")


In [ ]:
tips

In [ ]:
node_data = {'Node':['Pathology','BiologicalProcess','Abundance','Protein'],
       'Number':[3347,2068,1420,381]}
node = pd.DataFrame(node_data)
a = sns.barplot(x="Node", y="Number", data=node)
a.set(xlabel='Node',ylabel='Number',title= 'KG nodes in numbers (Total=7216)')

In [ ]:
nspace_data = {'Namespace':['ChEMBLAssay','ChEMBL','GOBP','GOMF','Protein','Disease','SideEffect','Reactome','MOA'],
       'Number':[862,558,774,537,381,36,3311,751,6]}
nspace = pd.DataFrame(nspace_data)
a = sns.barplot(x="Number", y="Namespace", data=nspace_data)
a.set(xlabel='Number',ylabel='Namespace',title= 'KG Namespace in numbers')